## Build phantom halo from CT results.
So far halo/galaxy brick from HM/GM have been the base of mk_gal routine. But phantom halo information only exist in tree_0_0_0.dat. Because all the basic information of all halos/galaxies are also stored in the tree output, it might be better to run mk_gal routine based on tree output only.

And, because mk_gal routine requires halo class, I need a function that returns a new halo instance with basic information from the tree output.

In [1]:
import numpy as np
import tree.halomodule as hmo
import tree.treemodule as tmo
import pickle

wdir = '/home/hoseung/Work/data/29172/'
tree_path = 'GalaxyMaker/Trees/'

In [4]:
from tree import treemodule
import tree.ctutils as ctu

nout_fi = 187
is_gal = True
try:    
    alltrees = pickle.load(open(wdir + tree_path + "extended_tree.pickle", "rb" ))
    print("Loaded an extended tree")
except:
    
    alltrees = treemodule.CTree()
    alltrees.load(filename= wdir + tree_path + 'tree_0_0_0.dat')
    # Fix nout -----------------------------------------------------
    nout_max = alltrees.data['nout'].max()
    alltrees.data['nout'] += nout_fi - nout_max
    print("------ NOUT fixed")
    alltrees.data = ctu.augment_tree(alltrees.data, wdir, is_gal=is_gal)
    print("------ tree data extended")
    pickle.dump(alltrees, open(wdir + tree_path + "extended_tree.pickle", "wb" ))


Loading Consistent Tree data from ASCII is done
------ NOUT fixed
------ tree data extended


In [5]:
tree_element = alltrees.data[1234]

In [15]:
import load
nout = 187
info = load.info.Info(nout=nout, base=wdir, load=True)

In [8]:
nout = tree_element['nout']
h = hmo.Halo(nout=nout, halofinder='HM', info=info, is_gal=True)
# required information
# mvir, x, y, z, rvir, vx, vy, vz, id
#

In [23]:
def halo_from_tree(tree_element, info):
    dtype_halo = [('id', '<i4'), ('m', '<f4'), ('mvir', '<f4'),
              ('r', '<f4'), ('rvir', '<f4'), 
              ('x', '<f4'), ('y', '<f4'), ('z', '<f4'),
              ('vx', '<f4'), ('vy', '<f4'), ('vz', '<f4')]
    
    cboxsize = 200.
    
    nout = tree_element['nout']
    h = hmo.Halo(nout=nout, halofinder='HM', info=info, is_gal=True)
    
    h.data = np.recarray(1, dtype=dtype_halo)
    h.nout = nout
    
    h.data['m'] = tree_element['m']
    h.data['mvir'] = tree_element['mvir']
    h.data['x'] = tree_element['x'] / cboxsize
    h.data['y'] = tree_element['y'] / cboxsize
    h.data['z'] = tree_element['z'] / cboxsize # Mpc/h -> code unit
    h.data['vx'] = tree_element['vx']
    h.data['vy'] = tree_element['vy']
    h.data['vz'] = tree_element['vz']
    h.data['r'] = tree_element['r'] # already in code unit
    h.data['rvir'] = tree_element['rvir'] / (cboxsize * 1000) # kpc/h -> code unit
    h.data['id'] = tree_element['Orig_halo_id']
    h.aexp = tree_element['aexp']
    
    return h

a = halo_from_tree(tree_element, info)

In [24]:
a.data

rec.array([ (232, 252920848.0, 756300032.0, 6.941806350369006e-05, 6.941999890841544e-05, 0.49209749698638916, 0.6785699725151062, 0.45732349157333374, 645.2000122070312, 213.88999938964844, -199.30999755859375)], 
          dtype=[('id', '<i4'), ('m', '<f4'), ('mvir', '<f4'), ('r', '<f4'), ('rvir', '<f4'), ('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('vx', '<f4'), ('vy', '<f4'), ('vz', '<f4')])